In [1]:
import matplotlib.pyplot as plt
import numpy as np
from read_dat import *
from scipy.optimize import curve_fit

def gaussian(x, mu, sigma, A):
    return (A*(1/(sigma*np.sqrt(2*np.pi)))*np.exp(-(1/2)*((x-mu)/sigma)**2))

In [2]:
calibration = [0.00010230476254145869, 0.012454430536400718]

In [3]:
dat_file = read_dat('../Data/AmBe.dat', align_method='max')


init complete


In [4]:
%matplotlib tk
# file.add_selections(L, S, mode='m')
dat_file.add_selections(mode='p', file='AmBe_cuts.csv')

Selections Imported
Polygons Created


In [5]:
file = open(f'AmBe_lst_out.csv', 'r')

reader = csv.reader(file)

head = next(reader, None)
print(head)
head = next(reader, None)
print(head)

L = []
S = []

for row in reader:
    L.append(float(row[0]))
    S.append(float(row[1]))

L = np.array(L)
S = np.array(S)

['../Data/AmBe channel 0, False events, cuts False']
['L [ch]', 'S[ch]', 'T (trigger) [us]', 'baseline', 'pulse height [bits]']


In [6]:
cut_L, cut_S = dat_file.select_events(L, S, cut_id=[2], inc=[1])
cut_L = calibration[0] * cut_L + calibration[1]

In [7]:
plt.hist2d(cut_L, cut_S, [256,256], norm=colors.LogNorm(vmin=1))
plt.xlabel('L (MeVee)')
plt.ylabel('S')
plt.colorbar(label='Counts')
plt.show()

In [8]:
all_S_hist, all_S_bins = np.histogram(cut_S[cut_L >= 0.1], bins='auto')

# plt.hist(cut_S[cut_L >= 3000], bins='auto')

plt.step(all_S_bins[:-1], all_S_hist)
plt.xlabel('S')
plt.ylabel('Counts')
plt.title('Histogram of PSD parameter S for AmBe source (L >= 0.1)')
plt.show()

# FoM by splitting simply along S value

In [9]:
# Fake splits
split_point_index = np.where(all_S_bins >= 0.61)[0][0]

neutron_fit = curve_fit(gaussian, all_S_bins[:split_point_index], all_S_hist[:split_point_index], [1, 1, 1])
gamma_fit = curve_fit(gaussian, all_S_bins[split_point_index:-1], all_S_hist[split_point_index:], [1, 1, 1])

print(neutron_fit[0])
print(gamma_fit[0])


[5.12587871e-01 5.88560019e-02 1.22787458e+03]
[6.81191514e-01 2.63903350e-02 8.08750117e+02]


In [10]:
FoM = np.abs(gamma_fit[0][0] - neutron_fit[0][0]) / (2.35 * gamma_fit[0][1] + 2.35 * neutron_fit[0][1])
print(f'Figure of Merit: {FoM}')

Figure of Merit: 0.8416341801266856


In [11]:
plt.step(all_S_bins[:-1], all_S_hist, label='data')

plt.plot(all_S_bins[:split_point_index], gaussian(all_S_bins[:split_point_index], *neutron_fit[0]), label='neutron fit')
plt.plot(all_S_bins[split_point_index:-1], gaussian(all_S_bins[split_point_index:-1], *gamma_fit[0]), label='gamma fit')

plt.xlabel('S')
plt.ylabel('Counts')
plt.title(f'Histogram of PSD parameter S for AmBe source (L >= 0.1 MeVee)\nFoM: {FoM:.5f}')
plt.legend()

plt.show()

# FoM splitting them by PSD cuts

In [12]:
gamma_L, gamma_S = dat_file.select_events(L, S, cut_id=[1], inc=[1])
gamma_L = calibration[0] * gamma_L + calibration[1]

In [13]:
gamma_S_hist = np.histogram(gamma_S[gamma_L >= 0.1], bins='auto')

plt.step(gamma_S_hist[1][:-1], gamma_S_hist[0])
plt.show()


In [19]:
neutron_L, neutron_S = dat_file.select_events(L, S, cut_id=[0], inc=[1], visual=True)
neutron_L = calibration[0] * neutron_L + calibration[1]

In [15]:
neutron_S_hist = np.histogram(neutron_S[neutron_L >= 0.1], bins='auto')

plt.step(neutron_S_hist[1][:-1], neutron_S_hist[0])
plt.show()


In [16]:
popt_g, pcov_g = curve_fit(gaussian, gamma_S_hist[1][:-1], gamma_S_hist[0], [1, 1, 1])
popt_n, pcov_n = curve_fit(gaussian, neutron_S_hist[1][:-1], neutron_S_hist[0], [1, 1, 1])

In [17]:
FoM_2 = np.abs(popt_g[0] - popt_n[0]) / (2.35 * popt_g[1] + 2.35 * popt_n[1])
print(f'Figure of Merit: {FoM_2}')

Figure of Merit: 0.9331560611008926


In [18]:
plt.step(gamma_S_hist[1][:-1], gamma_S_hist[0], label='gamma data')
plt.step(neutron_S_hist[1][:-1], neutron_S_hist[0], label='neutron data')

plt.plot(gamma_S_hist[1][:-1], gaussian(gamma_S_hist[1][:-1], *popt_g), label='gamma fit')
plt.plot(neutron_S_hist[1][:-1], gaussian(neutron_S_hist[1][:-1], *popt_n), label='neutron fit')

plt.xlabel('S')
plt.ylabel('Counts')
plt.title(f'Histogram of PSD parameter S for AmBe source (L >= 0.1 MeVee)\nFoM: {FoM_2:.5f}')
plt.legend()

plt.show()



# AmBe neutron L spectrum

In [20]:
plt.figure()
plt.hist(neutron_L, bins='auto', histtype='step')
plt.xlabel('L (MeVee)')
plt.ylabel('Counts')
plt.title('AmBe neutron light output spectrum')
plt.semilogy()
plt.show()